# Reprezentace matic a vektorů, základní operace lineární algebry

## Násobení matic a jeho očekávaná rychlost

Násobení matic vyžaduje $2 N^3$ aritmetických operací provedených nad $2 N^2$ číslech. Je ideální pro současné CPU, jejichž aritmetický výkon výrazně předbíhá rychlost paměti, mohou na něm dosáhnout téměř svého teoretického výkonu.
Tak to zkusme. 

In [ ]:
# Pomocná funkce pro generování náhodné matice
import random
def randmatrix(dim):
    return [ [ random.random() for i in range(dim) ] for j in range(dim) ]

In [ ]:
# velikost pokusné matice
N = 300

# vygenerujeme náhodné vstupy
A=randmatrix(N)
B=randmatrix(N)

In [ ]:
# budeme měřit, jak dlouho výpočet trvá
from time import time

### Vlastní implementace

Na místo označené v následující buňce `# TODO: C = AB` 
dopište vlastní jednoduchý kód pro maticové násobení

In [ ]:
# inicializace prázdného výstupu
C=[ [ 0. for i in range(N) ] for j in range(N) ]

start = time()

# TODO: C = AB
# ODOT


end = time()
measured_flops = 2 * N**3 / (end-start)

print(f'Násobení matic řádu {N} trvalo {end-start : .2f} s, tedy počítáme s výkonem { measured_flops/ 1e9 : .3f} GFLOPS')


### Očekávaná rychlost 

Na celkový výkon CPU pro aritmetické operace má vliv jeho frekvence a řada faktorů konkrétní architektury, zejména šířka slova aritmetické jednotky, tj. kolik operací vykonává současně (zpravidla 256 nebo 512 bitů, tj. 4 nebo 8 operací), počet cyklů potřebných k dokončení jedné instrukce (_cycles per instruction_), a přítomnost speciální instrukce _fused multiply add_, současného provedení součtu a násobení.

U dnešních CPU můžeme čekat **40-100 GFLOPS** pro jedno jádro.

In [ ]:
# linuxová magie: informace vyčteme z /proc/cpuinfo a zpracujeme

from subprocess import check_output
mhz = max([ float(f) for f in check_output(['grep','cpu MHz','/proc/cpuinfo']).split()[3::4] ])
flags = str(check_output("grep 'flags' /proc/cpuinfo | head -1",shell=True)).split()[1:]
avx = 128
if 'avx2' in flags: avx=256
if 'avx512f' in flags: avx=512
fma = 2 if 'fma' in flags else 1
cpi = .5 # XXX: pro všechny moderní CPU
flops_cpuinfo = mhz * 1e6 / cpi * avx/64 * fma

model = " ".join(str(check_output("grep 'model name' /proc/cpuinfo | head -1",shell=True)).split()[2:-1])

print(f'Očekávaný výkon jednoho jádra tohoto procesoru ({model}) je {flops_cpuinfo / 1e9 :.0f} GFLOPS')

### Srovnání

Pravděpodobně zjišťujejme, že přímočará implementace násobení matic v Pythonu je více než 1000x promalejší, než bychom u daného CPU očekávali.

Python je interpretovaný jazyk, když programátor napíše cyklus, provádí ho přímočaře jednu iteraci po druhé. To vůbec nedovolí, aby se CPU projevil (provedení více operací současně, maskování pomalého přístupu do paměti).

Pro skutečné numerické výpočty je tento přístup nepřijatelný. 

### Knihovna numpy

Místo přímočarého přístupu musíme využít knihoven, jejichž implementace možnosti CPU dokáže využít, `numpy` je zlatý standard. 

- Poskytuje datové typy, které reprezentují vektory, matice, tenzory, ...
- Operace nad nimi jsou implementovány efektivně s ohledem na možnosti CPU
- Formulací problému "vektorově" na tuto výpočetní efektivitu dosáhneme
- Viz https://numpy.org/doc/stable/

Obecně platí, že knihovnám vždy musíme umožnit pracovat s větším množstvím čísel současně, tedy pracovat explicitně s vektory, maticemi a tenzory, ne s jejich jednotlivými prvky.


In [ ]:
import numpy as np

In [ ]:
# zkusme větší matice
Nn=5000

In [ ]:
An = np.array(randmatrix(Nn))
Bn = np.array(randmatrix(Nn))

In [ ]:
An

In [ ]:
start=time()

# TODO: najděte v dokumentaci numpy (nebo se zeptejte umělé inteligence), jak se v numpy vynásobí matice: Cn = An Bn

Cn = An @ Bn

# ODOT

end=time()
measured_flops = 2 * Nn**3 / (end-start)

print(f'Násobení matic řádu {Nn} trvalo {end-start : .2f} s, tedy počítáme s výkonem { measured_flops/ 1e9 : .3f} GFLOPS')


### Srovnání výkonu numpy

V tomto případě bychom měli dosáhnout na alespoň 80% teoretického výkonu CPU. To už je považováno za velmi dobrý výsledek, 20% snadno padne na přístupy do paměti, interakci Pythonu s implementací numpy, náběh z úsporného režimu procesoru, interferenci s dalšími uživateli systému apod.

In [ ]:
measured_flops / flops_cpuinfo

## Univerzální funkce a vektorizace
`numpy` poskytuje základní operace lineární algebry (maticové násobení, skalární součin, ...), na ty postupně narazíme a dále **univerzální funkce**, které (opět efektivně) pracují na všech prvcích vektorů, matic, ...

### Tlumený oscilátor
(další triviální umělý příklad)

Časový průběh veličiny $y$ (ať je to co je to) je:
$$ y(t) = Y e^{-t/T} \sin(\omega t) $$

In [ ]:
Y = 42.
T = 12.3456
omega = 2. * np.pi 

In [ ]:
# absurdně mnoho vzorků t
num = int(1e7)
ts = np.linspace(start=0., stop=20., num=num)
ts

In [ ]:
# prostor pro výsledek
ys = np.zeros_like(ts)

In [ ]:
from math import exp,sin

In [ ]:
start = time()

# naivní implementace cyklem
for i,t in enumerate(ts):
    ys[i] = Y * exp(-t / T) * sin(omega * t)

end = time()

it_s = num / (end-start)

print(f'Počet iterací za sekundu: {it_s : .0f}, tj. { flops_cpuinfo / it_s : .0f} operací na iteraci (při teoretickém výkonu tohoto CPU)')
    

### Očekávaný výkon

Složitější funkce `sin` a `exp` jsou implementovány Taylorovým rozvojem, vyžadujeme přesnost na 16-20 desetinných míst, lze čekat orientačně 1000 základních aritmetických operací na každou, ale ne řádově více.

Problém je opět v programátorem napsaném cyklu.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# nakreslíme si výsledek, ať není nuda
plt.plot(ts,ys)
plt.show()

### Implementace v numpy

Najděte v dokumentaci, jak použíti funkce `sin` a `exp` z knihovny `numpy` a proveďte stejný výpočet bez explicitního cyklu.

In [ ]:
start = time()

# TODO: a teď totéž s numpy

y = Y * np.exp(-ts / T) * np.sin(omega * ts)

# ODOT

end = time()

it_s = num / (end-start)

print(f'Počet iterací za sekundu: {it_s : .0f}, tj. { flops_cpuinfo / it_s : .0f} operací na iteraci (při teoretickém výkonu)')



## Osy a broadcast

Numpy pracuje s pojmem _osa (axis)_, označuje pořadí indexu ve vícerozměrném poli. Není-li to jednoznačné, operace vyžadují specifikaci, podle které osy mají postupovat. Např.:

In [ ]:
A = np.random.uniform(5,size=(3,3)).astype(np.int32)
A

In [ ]:
np.sum(A,axis=None)   #default, všechny osy 

In [ ]:
np.sum(A,axis=0) # součty sloupců

In [ ]:
np.sum(A,axis=1) # součty řádků

Tam, kde si rozměry polí neodpovídají, `numpy` se snaží si je domyslet a vhodně doplnit opakováním, tzv. _broadcast_.

In [ ]:
x = np.array([1,2,3])

In [ ]:
x * 2

In [ ]:
x + np.array([2])
# tohle ale nejde:  x + np.array([2,2]) 

In [ ]:
A + x, (A + x) - A  # vektor je řádek, broadcastuje se na opakování po řádcích

In [ ]:
xt = x[:,np.newaxis] # sloupcový vektor zkonstruujeme "přidáním" druhé osy
xt

In [ ]:
A + xt, (A + xt) - A

In [ ]:
mx = -x
mx[:,np.newaxis] + x  # broadcast po obou osách